In [1]:
from pettingzoo.classic import rps_v2
from pettingzoo import AECEnv
import os
import openai
import time
from utils import write_to_file, ask
from pathlib import Path
from prompt import *

from dotenv import load_dotenv
load_dotenv()

True

In [2]:
# Indicate the LLM version that you want to use
# 'gpt-3.5' / 'gpt-3'
GPT_MODEL = 'gpt-3'
# max number of trials for each task
NUM_TRY = 6
# By default, the max number of tokens is 1400. In some tasks, the context limit of the language models is exceeded. Try to change the number of tokens in these cases.
MAX_TOKENS = 1400
# Output path
LOG_PATH = Path('./log/')
F_LM = Path('lm_log.txt')
F_GAME = Path('game_log.txt')

In [3]:
import os

def simulate(agents: any, env: AECEnv) -> dict[any, float]:
    env.reset()
    rewards = {agent: 0 for agent in env.possible_agents}
    
    for agent in agents.values():
        agent.reset()

    for agent_name in env.agent_iter():
        observation, reward, termination, truncation, info = env.last()
        rewards[agent_name] += reward
        obs_message = agents[agent_name].observe(
            observation, reward, termination, truncation, info
        )
        if termination or truncation:
            action = None
        else:
            action = agents[agent_name].act()
        print(f"Agen {agent_name} Action {action}")
        env.step(action)
    env.close()
    return rewards

Agent = None

for _ in range(1):
    get_define_agent_prompt = get_solution_prompt\
        .replace("<simulation code>", simulation_code)\
        .replace("<starter>", rps_starter)\
        .replace("<env info>", rps_info)
    print(get_define_agent_prompt)
    # agent_response = ask(get_define_agent_prompt, GPT_MODEL, MAX_TOKENS, LOG_PATH, F_LM)
    # define_agent_code = parse(agent_response)
    define_agent_code = """
    class Agent():
        def __init__(self, env, name):
            self.env = env
            self.name = name
        
        def reset(self):
            pass
            
        def observe(self, observation, reward, termination, truncation, info):
            pass
            
        def act(self):
            observation, reward, termination, truncation, info = self.env.last()
            return self.env.action_space(self.name).sample()
    """.strip()
    env = rps_v2.env(max_cycles=5, render_mode="human")
    exec(define_agent_code)
    agents = {
        name: Agent(env, name)
        for name in env.possible_agents
    }
    simulate(agents, env)
    # get_refinement_prompt = f"given the execution result: {info}"
    # refinement_response = 
    # (get_refinement_prompt)

# You are a given a task to write a game agent to interact with a pettingzoo environment.
Your agent should be able to interact with the environment by calling the following methods:
```python
from pettingzoo.classic import tictactoe_v3

# create the environment
env = tictactoe_v3.env()
agents = {}
# create the agent using the Agent code defined by you
agents[env.possible_agents[0]] = Agent(env)
# you will be playing against an opponent agent
agents[env.possible_agents[1]] = OpponentAgent(env, name)


def simulate(agents: any, env: AECEnv) -> dict[any, float]:
    env.reset()
    rewards = {agent: 0 for agent in env.possible_agents}
    
    for agent in agents.values():
        agent.reset()

    for agent_name in env.agent_iter():
        observation, reward, termination, truncation, info = env.last()
        rewards[agent_name] += reward
        obs_message = agents[agent_name].observe(
            observation, reward, termination, truncation, info
        )
        if termination o